In [1]:
from facebook_business.api import FacebookAdsApi
from facebook_business.adobjects.adaccount import AdAccount
import os
from datetime import datetime
from dotenv import load_dotenv

/Users/lroberts/bulkuploader/venv/lib/python3.8/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
load_dotenv()  # This loads the variables from .env

True

In [10]:
def get_audiences(ad_account_id):
    FacebookAdsApi.init(access_token=os.getenv('FACEBOOK_ACCESS_TOKEN'),
                        app_id=os.getenv('FACEBOOK_APP_ID'),
                        app_secret=os.getenv('FACEBOOK_APP_SECRET'))

    ad_account = AdAccount(ad_account_id)

    try:
        saved_audiences = ad_account.get_saved_audiences(fields=[
            'id',
            'name',
            'time_updated',
            # Add other fields as needed
        ])
        # Convert Cursor to list of dictionaries
        audience_list = [audience.export_all_data() for audience in saved_audiences]
        return audience_list
    except Exception as e:
        print(f'An error occurred: {e}')


In [11]:
# Replace 'act_453485805236160' with your actual ad account ID
audiences = get_audiences('act_453485805236160')
for audience in audiences:
    print(audience)

{'id': '23843860520770626', 'name': 'Broad', 'time_updated': '2022-04-15T16:54:31+0000'}
{'id': '23843863636030626', 'name': 'W/E/P/E', 'time_updated': '2022-04-15T16:52:51+0000'}
{'id': '23843866198050626', 'name': 'F/C', 'time_updated': '2022-04-15T16:53:38+0000'}
{'id': '23843866199350626', 'name': 'AI - Members', 'time_updated': '2021-03-22T16:41:27+0000'}
{'id': '23846903082240626', 'name': 'F/C (age cap)', 'time_updated': '2022-04-15T16:55:01+0000'}
{'id': '23846903083510626', 'name': 'Broad (age cap)', 'time_updated': '2022-04-15T16:55:51+0000'}
{'id': '23849433938990626', 'name': "Target (Test Feb '22)", 'time_updated': '2022-02-10T18:47:27+0000'}
{'id': '23849433941100626', 'name': "AI - CR (Test Feb '22)", 'time_updated': '2022-04-15T16:52:12+0000'}
{'id': '23849433942630626', 'name': "F/C (Test Feb '22)", 'time_updated': '2022-04-15T16:47:45+0000'}
{'id': '23849433943550626', 'name': "Broad (Test Feb '22)", 'time_updated': '2022-04-15T16:44:47+0000'}
{'id': '2384985655504062

In [17]:
def find_most_recent_audience(audiences, keywords):

    # Filter audiences based on keywords
    filtered_audiences = [aud for aud in audiences if any(keyword.lower() in aud.get('name', '').lower() for keyword in keywords)]

    if not filtered_audiences:
        return None, f"No audience found containing the keywords '{keywords}'."

    # Find the most recent audience
    most_recent_audience = max(filtered_audiences, key=lambda aud: datetime.strptime(aud.get('time_updated', '1900-01-01T00:00:00+0000'), '%Y-%m-%dT%H:%M:%S%z'))

    return most_recent_audience, None


In [18]:
# Simplified keyword lists
broad_keywords = ['broad']
wep_keywords = ['w/e/p/e', 'wep']
reach_keywords = ['reach', 'target area']

broad_audience, broad_error = find_most_recent_audience(audiences, broad_keywords)
wep_audience, wep_error = find_most_recent_audience(audiences, wep_keywords)
reach_audience, reach_error = find_most_recent_audience(audiences, reach_keywords)

In [19]:
for aud, error, category in zip([broad_audience, wep_audience, reach_audience], [broad_error, wep_error, reach_error], ['Broad', 'W/E/P/E', 'Reach or Target Area']):
    if error:
        print(error)
    else:
        print(f"Most recent '{category}' audience: ID - {aud['id']}, Name - {aud['name']}")


Most recent 'Broad' audience: ID - 23851961174370626, Name - Broad 18+ v3
Most recent 'W/E/P/E' audience: ID - 23851961172890626, Name - W/E/P/E + AI v3
Most recent 'Reach or Target Area' audience: ID - 23851891868640626, Name - Reach v3
